# OData Measurement and Test Result Queries with Digital Thread Services

This notebook demonstrates how to use OData queries to retrieve and analyze test measurements, results, and conditions from the NI Measurement Data Store. It shows hierarchical navigation through TestResults → Steps → Measurements.

## Prerequisites

**⚠️ Important:** Run the `publish_sample_data.ipynb` notebook first to create the sample test data that this notebook queries.

- NI Measurement Data Store running and accessible
- Python environment with the `ni.datastore` package
- Sample test data created by running `publish_sample_data.ipynb`

## Setup

Import the required libraries and create both metadata and data store clients.

In [1]:
from ni.datastore.data import DataStoreClient

data_store_client = DataStoreClient()

## Query Measurements

Measurements are the individual data points collected during test steps.

**Additional measurement query examples:**

```python
# Filter by measurement name
voltage_measurements = data_store_client.query_measurements("$filter=contains(Name,'Voltage')")
```

In [ ]:
from ni.datastore.data import Outcome
from ni.datastore.metadata import MetadataStoreClient

# Find failed measurements by checking outcome
print("\n1. Failed Measurements:")
failed_measurements = data_store_client.query_measurements("")
for measurement in failed_measurements:
    print(f"*** data type: {measurement.data_type}")
    measurement_name = measurement.measurement_name or "Unnamed Measurement"
    if measurement.start_date_time:
        print(f"  ❌ {measurement_name} - Failed at {measurement.start_date_time.strftime('%Y-%m-%d %H:%M:%S')}")
    else:
        print(f"  ❌ {measurement_name} - Failed at Unknown Time")

if not failed_measurements:
    print("  ✅ No failed measurements found!")

if len(failed_measurements) > 15:
    print(f"  ... and {len(failed_measurements) - 15} more measurements")

def print_measurement_with_outcome(measurement) -> str:
    name = measurement.measurement_name or "Unnamed Measurement"
    passed = measurement.outcome == Outcome.OUTCOME_PASSED
    failed = measurement.outcome == Outcome.OUTCOME_FAILED
    return f"{name} - {'✅ (Passed)' if passed else '❌ (Failed)' if failed else '❓ (Unknown)'}"

print("\n=== Voltage Measurements ===")
voltage_measurements = data_store_client.query_measurements("$filter=contains(Name,'Voltage')")
for measurement in voltage_measurements:
    print(f"  ⚡ {print_measurement_with_outcome(measurement)}")

print("\n=== Current Measurements ===")
current_measurements = data_store_client.query_measurements("$filter=contains(Name,'Current')")
for measurement in current_measurements:
    print(f"  🔌 {print_measurement_with_outcome(measurement)}")

print("\n=== Failed Measurements by Operator Alex Smith ===")
metadata_store_client = MetadataStoreClient()
for measurement in failed_measurements:
    if measurement.test_result_id:
        test_result = data_store_client.get_test_result(measurement.test_result_id)
        if test_result.operator_id:
            operator = metadata_store_client.get_operator(test_result.operator_id)
            operator_name = operator.operator_name
            if operator_name == "Alex Smith":
                print(f"  📋 {print_measurement_with_outcome(measurement)} - Operator: {operator_name}")


1. Failed Measurements:
  ❌ 1kHz Response - Failed at 2025-10-23 19:46:03
  ❌ 20kHz Response - Failed at 2025-10-23 19:46:03
  ❌ Supply Voltage - Failed at 2025-10-23 19:46:02
  ❌ Measure Input Voltage - Failed at 2025-10-23 19:45:56
  ❌ Left Channel DC Offset - Failed at 2025-10-23 19:46:02
  ❌ Right Channel DC Offset - Failed at 2025-10-23 19:46:02
  ❌ 75% Load Test - Failed at 2025-10-23 19:45:56
  ❌ Measure -12V Rail - Failed at 2025-10-23 19:45:56
  ❌ Calculate Input Power - Failed at 2025-10-23 19:45:56
  ❌ 1kHz THD+N Left - Failed at 2025-10-23 19:46:03
  ❌ Max Power Left - Failed at 2025-10-23 19:46:03
  ❌ SNR Right Channel - Failed at 2025-10-23 19:46:03
  ❌ No Load Test - Failed at 2025-10-23 19:45:56
  ❌ voltage_response - Failed at 2025-10-23 19:43:25
  ❌ voltage_response - Failed at 2025-10-23 19:43:25
  ❌ 50% Load Test - Failed at 2025-10-23 19:45:56
  ❌ Standby Current - Failed at 2025-10-23 19:46:02
  ❌ Full Load Test - Failed at 2025-10-23 19:45:56
  ❌ Configure DMM -

## Query Steps

Steps are the individual test phases within a test result. Each step can contain multiple measurements.

**Additional step query examples:**

```python
# Filter by step name
initialization_steps = data_store_client.query_steps("$filter=contains(Name,'Initialize')")
measurement_steps = data_store_client.query_steps("$filter=contains(Name,'Measure')")
```

In [ ]:
# Query all steps
all_steps = list(data_store_client.query_steps(""))

print(f"Found {len(all_steps)} steps total")
print("\nStep summary:")
for step in all_steps[:10]:  # Show first 10 steps
    step_name = step.step_name or "Unnamed Step"
    # Steps don't have a status field, so we'll skip showing status
    print(f"  🔧 {step_name}")

if len(all_steps) > 10:
    print(f"  ... and {len(all_steps) - 10} more steps")

print("\n=== Steps containing 'Voltage' ===")
voltage_steps = list(data_store_client.query_steps("$filter=contains(Name,'Voltage')"))
for step in voltage_steps:
    step_name = step.step_name or "Unnamed Step"
    # Steps don't have a status field, so we'll skip showing status
    print(f"  🔧 {step_name}")

## Query Published Conditions

Published conditions represent test conditions or environmental factors during testing.

**Additional condition query examples:**

```python
# Filter by condition name
temperature_conditions = data_store_client.query_conditions("$filter=contains(Name,'Temperature')")
humidity_conditions = data_store_client.query_conditions("$filter=contains(Name,'Humidity')")
```

In [ ]:
# Query all published conditions
# Query conditions with name including 'Temperature' or 'Pressure'
temp_and_pressure_conditions = list(data_store_client.query_conditions("$filter=contains(Name,'Temperature') or contains(Name,'Pressure')"))

# Get unique condition names and sort them
unique_condition_names = sorted(set(
    condition.condition_name if hasattr(condition, 'condition_name') else "Unnamed Condition"
    for condition in temp_and_pressure_conditions
))

if temp_and_pressure_conditions:
    print("\nConditions related to temp and pressure:")
    for condition_name in unique_condition_names[:10]:  # Show first 10 conditions
        name = condition_name
        print(f"  🌡️ {name}")
        
    if len(temp_and_pressure_conditions) > 10:
        print(f"  ... and {len(temp_and_pressure_conditions) - 10} more conditions")
else:
    print("No published conditions found in the sample data")

## Close the client

In [ ]:
data_store_client.close()